In [1]:
import pyspark

In [2]:
spark = pyspark.sql.SparkSession.builder.appName('test').getOrCreate()

In [3]:
#Ensure we are using the right kernel
print (spark.version)

2.3.0


In [4]:
import os
import sh
import shutil
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
from matplotlib import colors

In [5]:
local_working_data_dir = "/Users/adeshghadge/Downloads/BigDataProject"

In [6]:
tweeter_data = spark.read.json('file:///' + local_working_data_dir + '/tweets_cons.json')

In [131]:
#tweets_text = tweeter_data.select(["id", "text"])
filter_keywords = \
["uchicago", "university of chicago",\
"polskycenter", "hyde park campus", \
"pritzker school of medicine"
"polsky center", "maroonmade", "maroon made", "chicago maroon", "pritzker.uchicago",\
"harris school of public policy", "chicago booth", "booth school of business", \
"graham school of continuing liberal and professional studies", "norris university center", \
"kellogg school of management", "feinberg school of medicine",\
"mccormick school of engineering and applied sciences", \
"henry and leigh bienen school of music", "bienen school of music", \
"harvard", "harvard crimson", "northwestern university", "willie the wildcat", "northwestern wildcat",\
"michigan medicine", "the michigan daily", "the michigan every three weekly", "the michigan times", \
"pritzker school of law", "university of michigan", "umich", "michigan wolverines", \
"the chicago maroon", "the daily northwestern", \
"ross school of business"]

filter_keywords2 = \
["uchicago", "university of chicago",\
 "maroonmade", "maroon made", "chicago maroon", 
 "chicago booth", "booth school of business", \
"graham school of continuing liberal and professional studies", \
"kellogg school of management", "feinberg school of medicine",\
"mccormick school of engineering and applied sciences", \
"harvard", "harvard crimson", "northwestern university", "willie the wildcat", "northwestern wildcat",\
"michigan medicine", "the michigan daily", "the michigan every three weekly", "the michigan times", \
"university of michigan", "umich", "michigan wolverines", \
"the chicago maroon", "the daily northwestern", \
"ross school of business"]

filter_keywords3 = \
["uchicago", "university of chicago",\
 "maroonmade", "maroon made", "chicago maroon", 
 "chicago booth", "booth school of business", \
"graham school of continuing liberal and professional studies", \
"kellogg school of management", \
"harvard", "harvard crimson", "northwestern university", "willie the wildcat", "northwestern wildcat",\
"university of michigan", "umich", "michigan wolverines", \
"ross school of business"]

filter_keywords4 = \
["uchicago", "university of chicago"]

In [146]:
def findString(tweet, retweet, extended_tweet):
    for filterWord in filter_keywords4:
        if((tweet is not None and tweet.lower().find(filterWord) >= 0) or \
           (retweet is not None and retweet.lower().find(filterWord) >= 0) or \
           (extended_tweet is not None and extended_tweet.lower().find(filterWord) >= 0)):
            return True
            
    return False;    

findUdf = udf(findString)

In [147]:
tweets_filtered = tweeter_data\
                    .filter("id is not null and ltrim(rtrim(id)) <> '' and ((text is not null and ltrim(rtrim(text)) <> '') \
                    or (retweeted_status_text is not null and ltrim(rtrim(retweeted_status_text)) <> '') \
                    or (extended_tweet_full_text is not null and ltrim(rtrim(extended_tweet_full_text)) <> ''))")\
                    .withColumn("relevantTweet", findUdf(col("text"),\
                                                         col("retweeted_status_text"),\
                                                         col("extended_tweet_full_text")))

In [109]:
tweeter_data.count()

1361524

In [148]:
tweets_filtered.filter("relevantTweet == True").count()

143463

In [149]:
tweets_filtered.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- extended_tweet_full_text: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- favorited: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- is_quote_status: boolean (nullable = true)
 |-- lang: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- place_id: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_place_type: string (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- retweeted_status_created_at: string (nullable = true)
 |-- retweeted_status_favorite_count: long (nullable = true)
 |-- retweeted_status_favorited: boolean (nullable = true)
 |-- retweeted_status_id: long (nullable = true)
 |-- retweeted_status_lang: string (nullable = true)
 |-- retweeted_status_place_country: string (nullable = true)
 |-- retweeted_status_place_id: string (nullable = true)
 |-- retweeted_status_place_name: s

In [150]:
tweets_filtered.groupby("user_id", "user_screen_name").agg(count("user_id"))\
    .withColumnRenamed("count(user_id)", "TweetCountPerUser")\
    .orderBy("TweetCountPerUser", ascending = False).show(10)

+------------------+----------------+-----------------+
|           user_id|user_screen_name|TweetCountPerUser|
+------------------+----------------+-----------------+
|880005795974217728|        CMasden7|             7734|
|         859728522|       jurnskeff|             1683|
|        2376944491|      SmartSanta|             1363|
|         417357386|     uchicagogsu|              946|
|        2828951115| ghostwriterREAL|              928|
|         131144285|        UChicago|              888|
|        2790314135| truthsearch1957|              845|
|        2645320447|    harvardedits|              816|
|        2809834164|      infoguy411|              813|
|          20270494|  ChicagoMaroons|              739|
+------------------+----------------+-----------------+
only showing top 10 rows



In [44]:
tweeter_data.groupby("user_id", "user_screen_name").agg(sum("retweeted_status_retweet_count"))\
    .withColumnRenamed("sum(retweeted_status_retweet_count)", "RetweetCountPerUser")\
    .orderBy("RetweetCountPerUser", ascending = False).show(10)

+------------------+----------------+-------------------+
|           user_id|user_screen_name|RetweetCountPerUser|
+------------------+----------------+-------------------+
|        3249390261|       olanaaaaa|             543431|
|         347975630|       yxngebony|             543015|
|         329145208|    wolf_haley69|             540797|
|        2992083703|         _nnajib|             519916|
|         507494850|      momacrosby|             436693|
|878467860628111360|  waveofchlorine|             416349|
|        4873869610|       relyung13|             415264|
|         704557153|        ASAPAIMA|             384358|
|        1184579304|     valeria_ggz|             375239|
|742409509679702016|        Haylo_24|             374006|
+------------------+----------------+-------------------+
only showing top 10 rows



In [47]:
tweeter_data.filter("place_name is not null and ltrim(rtrim(place_name)) <> ''")\
                    .groupby("user_id", "user_screen_name", "place_name").agg(count("user_id"))\
    .withColumnRenamed("count(user_id)", "TweetCountPerUser")\
    .orderBy("TweetCountPerUser", ascending = False).show(10)

+----------+----------------+-------------------+-----------------+
|   user_id|user_screen_name|         place_name|TweetCountPerUser|
+----------+----------------+-------------------+-----------------+
|4715692053|     MikeWebbNow|Bailey's Crossroads|              294|
|4715692053|     MikeWebbNow|          Arlington|              280|
|1901454811|        HDee1967|             Zurich|              204|
|  73717355|  tmj_il_hsptlst|            Harvard|              199|
| 582140844|   sunnyherring1|       Pennsylvania|              110|
|   4186351|        dahuntsr|          Ann Arbor|              109|
| 289420613|      naomirwolf|          Manhattan|               76|
|4715692053|     MikeWebbNow|         Alexandria|               72|
|   1356241|       shazrasul|            Chicago|               72|
|2607703608|        A_Rockas|     City of London|               70|
+----------+----------------+-------------------+-----------------+
only showing top 10 rows



In [63]:
from datetime import datetime
import pytz
from dateutil.parser import parse
import pandas as pd

In [86]:
def convertToDate(dateVal):
    if(dateVal is not None):
        return (datetime.strptime(datetime.strftime(datetime.strptime(dateVal, '%a %b %d %H:%M:%S +0000 %Y'),\
                                    '%b %d,%Y %H'), '%b %d,%Y %H') - datetime(1970, 1, 1))\
    .total_seconds()#pd.to_datetime(dateVal)

udf_dateconvert = udf(convertToDate)   

In [87]:
tweeter_data.filter("created_at is not null and ltrim(rtrim(created_at)) <> ''")\
.withColumn("created_at2", (udf_dateconvert(col("created_at"))))\
.select(["created_at", "created_at2"])\
.groupby("created_at2").agg(count("created_at2"))\
.orderBy("count(created_at2)", ascending = False)\
.show(5)

+-----------+------------------+
|created_at2|count(created_at2)|
+-----------+------------------+
|1.5131844E9|              7359|
|1.5131808E9|              7246|
| 1.513188E9|              7061|
|1.5131988E9|              6898|
|1.5131772E9|              6851|
+-----------+------------------+
only showing top 5 rows



In [88]:
tweeter_data.filter("created_at is not null and ltrim(rtrim(created_at)) <> ''")\
.withColumn("created_at2", (udf_dateconvert(col("created_at"))))\
.select(["user_id", "user_screen_name", "created_at", "created_at2"])\
.groupby("user_id", "user_screen_name", "created_at2").agg(count("created_at2"))\
.orderBy("count(created_at2)", ascending = False)\
.show(5)

+------------------+----------------+-----------+------------------+
|           user_id|user_screen_name|created_at2|count(created_at2)|
+------------------+----------------+-----------+------------------+
|         237490098|    miguelencita|1.5217308E9|               116|
|        1098685988|         J4J_USA|1.5066324E9|               101|
|880005795974217728|        CMasden7|1.5019524E9|                91|
|811092358574080000|  sanjanakharka1|1.5011352E9|                85|
|        1494071780|       Erik_H526|1.5212556E9|                84|
+------------------+----------------+-----------+------------------+
only showing top 5 rows



In [103]:
tweeter_data.filter("user_screen_name == 'miguelencita'").select("text").show(1, False)

+---------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                   |
+---------------------------------------------------------------------------------------------------------------------------------------+
|@mmikel6 @MOSupremeCt @USSupremeCourt @TheJusticeDept @CivilRights @Cornell @Harvard @Yale @Missouri @MOcourts… https://t.co/Y2g4Vf4moH|
+---------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [84]:
#pd.to_datetime("Wed Dec 13 15:15:02 +0000 2017")
datetime.strptime(datetime.strftime(datetime.strptime("Wed Dec 13 15:15:02 +0000 2017", '%a %b %d %H:%M:%S +0000 %Y'),\
                                    '%b %d,%Y %H'), '%b %d,%Y %H') \
- datetime(1970, 1, 1)

datetime.datetime(2017, 12, 13, 15, 0)

In [14]:
tweeter_data.filter("retweeted_status_text is not null and ltrim(rtrim(retweeted_status_text)) <> ''")\
.select(["id", "text", "retweeted", "retweeted_status_retweeted", "retweeted_status_text"]).show()

+------------------+--------------------+---------+--------------------------+---------------------+
|                id|                text|retweeted|retweeted_status_retweeted|retweeted_status_text|
+------------------+--------------------+---------+--------------------------+---------------------+
|940963279932919811|RT @AyrtonLittle:...|    false|                     false| All the hard work...|
|940963285167374336|RT @AyrtonLittle:...|    false|                     false| All the hard work...|
|940963286245355520|RT @AyrtonLittle:...|    false|                     false| All the hard work...|
|940963286304088064|RT @AyrtonLittle:...|    false|                     false| All the hard work...|
|940963286467592192|RT @AyrtonLittle:...|    false|                     false| All the hard work...|
|940963286639693824|RT @AyrtonLittle:...|    false|                     false| All the hard work...|
|940963287423963136|RT @AyrtonLittle:...|    false|                     false| All the hard

In [9]:
tweeter_data.filter("retweeted_status_text is not null and ltrim(rtrim(retweeted_status_text)) <> ''")\
.count()

963477

In [11]:
tweeter_data.filter("retweeted_status_retweeted == true")\
.count()

0

In [9]:
tweeter_data.count()

1361524

In [12]:
tweeter_data.filter("retweeted_status_text is null or ltrim(rtrim(retweeted_status_text)) == ''")\
.select(["id", "text", "extended_tweet_full_text", "retweeted", "retweeted_status_text"]).show()

+------------------+--------------------+------------------------+---------+---------------------+
|                id|                text|extended_tweet_full_text|retweeted|retweeted_status_text|
+------------------+--------------------+------------------------+---------+---------------------+
|940963372383731713|#TrueBlackExcelle...|                    null|    false|                 null|
|940963375156289537|Meet the young wo...|                    null|    false|                 null|
|940963541594591237|Former University...|                    null|    false|                 null|
|940963737061744641|Strategy: Harvard...|    Strategy: Harvard...|    false|                 null|
|940963859216596998|@lilpump graduate...|                    null|    false|                 null|
|940963943509577728|.@UChicago what s...|    .@UChicago what s...|    false|                 null|
|940964027651543041|MUST-WATCH: Presi...|                    null|    false|                 null|
|940964073